# General Overview of the Reconstruction Performances for a Moving Absorbing Boundary Condition

**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

**Do we want the matplotlib interactive magicness?**

In [1]:
%matplotlib widget

**Library Import**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn
import itertools
# For parallelization
from joblib import Parallel, delayed

import nekhoroshev_tools as nt

In [3]:
def afpt(I_min, I_max, I_star, exponent, c=None):
    if c is None:
        c = nt.standard_c(0.0, I_max, I_star, exponent)
    return scipy.integrate.quad(
        lambda x: 2*x/nt.D(x, I_star, exponent, c=c),
        I_min,
        I_max
    )[0]

## First, we need to define a realistic initial distribution

It's very simple and direct: if we decide to work in normalized emittance variables, and we work with a gaussian beam distribution with it's characteristic emittance, we have that
$$\rho_0(I) = e^{-I}$$
Where $I$, from now on, is expessed in sigma units.

In [4]:
def rho_0(I, damping_position=np.nan, l=np.nan):
    if np.isnan(damping_position) or np.isnan(l):
        return np.exp(-I)
    else:
        return np.exp(-I) / (1 + np.exp((I - damping_position)/l))

## Setting up a regular scenario

In [5]:
I_damping = 1.9
I_max = 2.0
I_star = 8.0
k = 0.33
exponent = 1/(2*k)

c = nt.standard_c(0.0, I_max, I_star, exponent)

In [6]:
cn_sampling = 10000
I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

cn_time_steps = 50
dt = nt.current_peak_time(I_damping, I_max, I_star, exponent, c)/cn_time_steps

# "Very fast" tracking

# "Long enough" tracking

## Working in the semi-stable region

Given $I_*=10.0$, we will firstly analyze the semi-stable region $[5.0-8.0]$ where we have the beginning of the saturation phase for $D(I)$.

In [7]:
steps_per_sample = 5

steps_per_iteration = 5000
iterations = 6
macro_iterations = 4
long_track_steps = steps_per_iteration * iterations * (macro_iterations + 1)

In [8]:
afpt(
    5.4,
    5.6,
    I_star,
    exponent,
    c=nt.standard_c(0.0, I_max, I_star, exponent)
)

4.433044361540415e-07

## How's the regular current for a single, constant $I_a$?

In [9]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))
data_0 = engine.get_data_with_x()

times_long, current_long = engine.current(long_track_steps, steps_per_sample, False)
data_long = engine.get_data_with_x()

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)
100%|██████████| 150000/150000 [02:33<00:00, 978.52it/s] 


**That's how our distribution (slowly) evolves...**

(In order to see things it's necessary to work with matplotlib interactive mode and zoom a lot)

In [10]:
plt.figure()
plt.plot(data_0[0], data_0[1], label="Initial condition")
plt.plot(data_long[0], data_long[1], label="After iterations")
plt.axvline(I_max, color="black", label="$I_a$")
plt.xlabel("$I$")
plt.ylabel("$\\rho$")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**And here's the current!**

In [11]:
plt.figure()
plt.plot(times_long, current_long)
plt.xlabel("$t$")
plt.ylabel("$J$")
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Experimenting with a bit of dancing!

In [12]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

t_absolute_out = []
t_relative_out = []
c_out = []
I_max_list = []
set_list = []

t_absolute_out_partial = []
t_relative_out_partial = []
c_out_partial = []
I_max_list_partial = []
set_list_partial = []

for j in tqdm(range(macro_iterations + 1)):
    if j != 0:
        engine.move_barrier_backward(0.1 * iterations)

    for i in tqdm(range(iterations)):
        time, current = engine.current(steps_per_iteration, steps_per_sample)
        if len(t_absolute_out) > 0:
            absolute_time = time + t_absolute_out[-1][-1]
        else:
            absolute_time = time.copy()
        t_relative_out.append(time)
        t_absolute_out.append(absolute_time)
        c_out.append(current)
        I_max_list.append(engine.I_max)
        set_list.append(i)
        if i != 0:
            t_relative_out_partial.append(time)
            t_absolute_out_partial.append(absolute_time)
            c_out_partial.append(current)
            I_max_list_partial.append(engine.I_max)
            set_list_partial.append(i)
        engine.move_barrier_forward(0.1)

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for i in range(len(t_absolute_out)):
    ax2.plot(
        [t_absolute_out[i][0],t_absolute_out[i][-1], t_absolute_out[i][-1]],
        [I_max_list[i],I_max_list[i],(I_max_list[i+1] if i+1 != len(I_max_list) else I_max_list[i])],
        c="black", alpha=0.5
    )
    ax1.plot(t_absolute_out[i], c_out[i])
    if i != len(t_absolute_out) - 1:
        ax1.plot(
            [t_absolute_out[i][-1], t_absolute_out[i][-1]],
            [c_out[i][-1], c_out[i+1][0]],
            c="black"
        )

ax1.plot(times_long, current_long, c="C0", linestyle="dashed")

ax1.set_yscale("log")
ax1.set_ylabel("Current")
ax1.set_xlabel("time")
ax2.set_ylabel("Barrier position")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Barrier position')

In [14]:
fig, ax1 = plt.subplots()

slice_size = len(current_long) // len(t_absolute_out)

c_normalized_out = []
c_normalized_out_partial = []

ax2 = ax1.twinx()
for i in range(len(t_absolute_out)):
    ax2.plot(
        [t_absolute_out[i][0],t_absolute_out[i][-1], t_absolute_out[i][-1]],
        [I_max_list[i],I_max_list[i],(I_max_list[i+1] if i+1 != len(I_max_list) else I_max_list[i])],
        c="black", alpha=0.5
    )
    ax1.plot(t_absolute_out[i], c_out[i]/(current_long[slice_size * i:slice_size * (i+1)]))
    c_normalized_out.append(c_out[i]/(current_long[slice_size * i:slice_size * (i+1)]))
    if i != len(t_absolute_out) - 1:
        ax1.plot(
            [t_absolute_out[i][-1], t_absolute_out[i][-1]],
            np.array([c_out[i][-1], c_out[i+1][0]])/current_long[slice_size*(i+1)],
            c="black"
        )

#ax1.plot(times_long, current_long, c="C0", linestyle="dashed")

#ax1.set_yscale("log")
ax1.set_ylabel("`Normalized` Current")
ax1.set_xlabel("time")
ax2.set_ylabel("Barrier position")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Barrier position')

In [15]:
plt.figure()
for i in range(1, len(c_normalized_out)):
    if i%6==1 and i != 1:
        plt.plot(t_absolute_out[0], c_out[i], c="C0")
    elif i%6==2 and i != 2:
        plt.plot(t_absolute_out[0], c_out[i], c="C1")
    elif i%6==3 and i != 3:
        plt.plot(t_absolute_out[0], c_out[i], c="C2")
    elif i%6==4 and i != 4:
        plt.plot(t_absolute_out[0], c_out[i], c="C3")
    elif i%6==5 and i != 5:
        plt.plot(t_absolute_out[0], c_out[i], c="C4")
    else:
        pass
    
plt.xscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure()
for i in range(1, len(c_normalized_out)):
    if i%6==1 and i != 1:
        plt.plot(t_absolute_out[0], c_normalized_out[i], c="C0")
    elif i%6==2 and i != 2:
        plt.plot(t_absolute_out[0], c_normalized_out[i], c="C1")
    elif i%6==3 and i != 3:
        plt.plot(t_absolute_out[0], c_normalized_out[i], c="C2")
    elif i%6==4 and i != 4:
        plt.plot(t_absolute_out[0], c_normalized_out[i], c="C3")
    elif i%6==5 and i != 5:
        plt.plot(t_absolute_out[0], c_normalized_out[i], c="C4")
    else:
        pass
    
plt.axvline(afpt(I_max_list[0], I_max_list[1], I_star, exponent, c), c="C0")
plt.axvline(afpt(I_max_list[1], I_max_list[2], I_star, exponent, c), c="C1")
plt.axvline(afpt(I_max_list[2], I_max_list[3], I_star, exponent, c), c="C2")
plt.axvline(afpt(I_max_list[3], I_max_list[4], I_star, exponent, c), c="C3")
plt.axvline(afpt(I_max_list[4], I_max_list[5], I_star, exponent, c), c="C4")

plt.axhline(0.96, c="grey")
plt.axhline(0.965, c="grey")
plt.axhline(0.97, c="grey")

plt.xscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
x_list = np.linspace(4.0, 5.2, 1000)

def weirder_func(x_list):
    return (
        1 - np.array([afpt(x, 5.2, 8.0, exponent, nt.standard_c(0.0, 5.0, 8.0, exponent)) for x in x_list])/afpt(5.0, 5.2, 8.0, exponent, nt.standard_c(0.0, 5.0, 8.0, exponent))
    )

plt.figure()
plt.plot(x_list, [afpt(x, 5.2, 8.0, exponent, nt.standard_c(0.0, 5.0, 8.0, exponent)) for x in x_list])
#plt.plot(x_list, weirder_func(x_list))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def weird_func(t, k):
    return -(np.exp(-k*t)-1)

plt.figure()

popt, pcov = scipy.optimize.curve_fit(
    weird_func,
    t_absolute_out[2],
    c_normalized_out[2],
    p0=5
)
plt.plot(t_absolute_out[0], c_normalized_out[2])
#plt.yscale("log")
plt.plot(t_absolute_out[0], weird_func(t_absolute_out[0], popt[0]))
plt.plot(t_absolute_out[0], weird_func(t_absolute_out[0], 3))
popt

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([9.8468968])

In [19]:
plt.figure()

plt.plot(t_absolute_out[0], c_normalized_out[2])
#plt.yscale("log")
plt.plot(t_absolute_out[0], weird_func(t_absolute_out[0], popt[0]))
popt

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([9.8468968])

In [20]:
plt.figure()
plt.plot(t_absolute_out[0], c_normalized_out[2])
plt.axvline(afpt(
    5.2,
    5.4,
    I_star,
    exponent,
    c=nt.standard_c(0.0, I_max, I_star, exponent)
))

plt.plot(t_absolute_out[0], c_normalized_out[3])
plt.axvline(afpt(
    5.4,
    5.6,
    I_star,
    exponent,
    c=nt.standard_c(0.0, I_max, I_star, exponent)
), c="C1")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
plt.figure()
plt.scatter(
    I_max_list_partial,
    [t_absolute_out_partial[i][np.argmax(c_out_partial[i])] - (t_absolute_out[i-1][-1] if i != 0 else 0) for i in range(len(t_absolute_out_partial))]
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …